In [127]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv("Invoice_20201220.csv", sep="|", encoding="latin-1")
df.head()

,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,12345_GEO (0001BB),1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,12345_GEO (0001BB),1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,12345_GEO (0001BB),1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,12345_GEO (0001BB),1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,12345_GEO (0001BB),1300082549,01/05/2018,31/05/2018,2 574,"333,37"


###  1) Le nom d'une antenne a un format bien précis : il est toujours composé de 4 chiffres et de deux lettres. On te demande de retirer tout autre caractère de la colonne "SITE_NAME".

In [128]:
# 1) Le nom d'une antenne a un format bien précis : il est toujours composé de 4 chiffres et de deux lettres. On te demande de retirer tout autre caractère de la colonne "SITE_NAME".
df['SITE_NAME'] = df['SITE_NAME'].str[11:17]

def clean_site_name(site_name):
    result = re.search("\d{4}[a-zA-Z]{2}", site_name)
    if result is None:
        return np.NaN
    else:
        return site_name
    return result

df["SITE_NAME"] = [clean_site_name(site_name) for site_name in df["SITE_NAME"]]
df

,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,0001BB,1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,0001BB,1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,0001BB,1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,0001BB,1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,0001BB,1300082549,01/05/2018,31/05/2018,2 574,"333,37"
...,...,...,...,...,...,...
51427,0632KO,FV1/055100030,30/08/2019,31/12/2019,3 028,"397,18"
51428,0632KO,055100058,01/01/2020,31/03/2020,2 163,"283,57"
51429,0632KO,055100113,01/04/2020,30/06/2020,2 160,"312,55"
51430,0632KO,055100168,01/07/2020,30/09/2020,2 230,"294,07"


### 2) Suppresion de invoice name 

In [129]:
df = df.drop(['INVOICE_NAME'], axis = 1)

### 3) Il y a des antennes pour lesquelles on a des factures mensuelles tandis que pour d'autres, on les reçoit moins régulièrement (trimestre, semestre, année). Ajoute une colonne à ton tableau qui calcule le nombre de jours entre le début de la période de facturation et la fin de la période de facturation. 

In [130]:
df["BEGIN"] = pd.to_datetime(df["BEGIN"], format="%d/%m/%Y")
df["END"] = pd.to_datetime(df["END"], format="%d/%m/%Y")
df.head()

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,0001BB,2018-01-01,2018-01-31,2 447,"316,46"
1,0001BB,2018-02-01,2018-02-28,2 211,"286,82"
2,0001BB,2018-03-01,2018-03-31,2 450,"317,32"
3,0001BB,2018-04-01,2018-04-30,2 431,"315,00"
4,0001BB,2018-05-01,2018-05-31,2 574,"333,37"


In [134]:
df['new'] = df["END"] - df["BEGIN"] 

### 4) A partir de cette nouvelle colonne, crée une autre colonne qui nous donne la consommation journalière moyenne de la période.

In [132]:
# df['KWH_PERIOD'] = df['KWH_PERIOD'].str.replace(' ', '').astype(int)
# df['new'] = df['new'].astype('timedelta64[D]').astype(int)

KeyError: 'new'

In [135]:
df['conso_mean'] = df['KWH_PERIOD'] / (df['new'].astype('timedelta64[D]').astype(int))

In [136]:
df['conso_mean'] = df['conso_mean'].round(decimals = 2)

### 5) Merci d'ajouter également une colonne qui donne, pour chaque antenne, le nombre de factures dont on dispose au total. Pas grave si l'information se répète à chaque ligne pour une même antenne.

In [137]:
df['freq_site'] = df.groupby('SITE_NAME')['SITE_NAME'].transform('count')

### 6) Et information bonus si tu y arrives : les équipes aimeraient beaucoup avoir une liste de toutes les antennes avec, pour chaque antenne, le coefficient de variation de la consommation. Cela permettra d'avoir une idée, pour chaque antenne, de la dispersion de sa consommation d'énergie. Coefficient de variation = (Écart-type / Moyenne) * 100

In [141]:
df.groupby('SITE_NAME')["KWH_PERIOD"].std()/df.groupby('SITE_NAME')["KWH_PERIOD"].mean()*100

SITE_NAME
0001BB      7.412620
0001BR      6.006796
0001KO     11.163076
0002BB      7.507342
0002BR      8.952184
             ...    
9513BB      0.376722
9515BB    117.872141
9516BB      1.812700
9517BB      0.055898
9518BB      0.311913
Name: KWH_PERIOD, Length: 2402, dtype: float64

In [156]:
df['CV'] = df["KWH_PERIOD"].std()/df["KWH_PERIOD"].mean()*100

In [157]:
df.head()

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,new,conso_mean,freq_site,CV
0,0001BB,2018-01-01,2018-01-31,2447,"316,46",30 days,81.57,35.0,482.321711
1,0001BB,2018-02-01,2018-02-28,2211,"286,82",27 days,81.89,35.0,482.321711
2,0001BB,2018-03-01,2018-03-31,2450,"317,32",30 days,81.67,35.0,482.321711
3,0001BB,2018-04-01,2018-04-30,2431,"315,00",29 days,83.83,35.0,482.321711
4,0001BB,2018-05-01,2018-05-31,2574,"333,37",30 days,85.80,35.0,482.321711


### 7) Enfin, tu dois exporter ces deux fichiers obtenus pour pouvoir l'envoyer aux équipes de l'opérateur et leur demander si c'est bien cela qu'ils attendaient de toi.

In [158]:
df.to_csv('energizes_new.csv')